In [2]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

from nltk.corpus import stopwords

from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

In [21]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [3]:
df=pd.read_csv("metadata.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309325 entries, 0 to 309324
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   cord_uid          309325 non-null  object 
 1   sha               113377 non-null  object 
 2   source_x          309325 non-null  object 
 3   title             309244 non-null  object 
 4   doi               190544 non-null  object 
 5   pmcid             116949 non-null  object 
 6   pubmed_id         167973 non-null  object 
 7   license           309325 non-null  object 
 8   abstract          220357 non-null  object 
 9   publish_time      309154 non-null  object 
 10  authors           299284 non-null  object 
 11  journal           289081 non-null  object 
 12  mag_id            0 non-null       float64
 13  who_covidence_id  103399 non-null  object 
 14  arxiv_id          4002 non-null    object 
 15  pdf_json_files    113377 non-null  object 
 16  pmc_json_files    85

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sushil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [57]:
# get stop words from nltk
stopWords = stopwords.words('english')

# pre processing data
def cleanData(sentence):
    processedList = ""
    
    # convert to lowercase, ignore all special characters - keep only alpha-numericals and spaces (not removing full-stop here)
    sentence = re.sub(r'[^A-Za-z0-9\s.]',r'',str(sentence).lower())
    sentence = re.sub(r'\n',r' ',sentence)
    
    # remove stop words
    sentence = " ".join([word for word in sentence.split() if word not in stopWords])
    
    return sentence

In [58]:
cleanData(df['abstract'][0])

'objective retrospective chart review describes epidemiology clinical features patients cultureproven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia. methods patients positive m. pneumoniae cultures respiratory specimens january december identified microbiology records. charts patients reviewed. results patients identified . required admission. infections . communityacquired. infection affected age groups common infants . preschool children .. occurred yearround common fall spring . threequarters patients . comorbidities. twentyfour isolates associated pneumonia upper respiratory tract infections bronchiolitis. cough . fever malaise . common symptoms crepitations wheezes common signs. patients pneumonia crepitations . bronchial breathing. immunocompromised patients likely nonimmunocompromised patients present pneumonia versus p .. patients pneumonia . uneventful recovery . recovered following complications . died pneumoniae infection . died due 

In [59]:
# clean data 
df['abstract'] = df['abstract'].map(lambda x: cleanData(x))

In [60]:
df['title'] = df['title'].map(lambda x: cleanData(x))

In [61]:
df[['title','abstract']].head()

,title,abstract
0,clinical features cultureproven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia,objective retrospective chart review describes epidemiology clinical features patients cultureproven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia. methods patients positive m. pneumoniae cultures respiratory specimens january december identified microbiology records. charts patients reviewed. results patients identified . required admission. infections . communityacquired. infection affected age groups common infants . preschool children .. occurred yearround common fall spring . threequarters patients . comorbidities. twentyfour isolates associated pneumonia upper respiratory tract infections bronchiolitis. cough . fever malaise . common symptoms crepitations wheezes common signs. patients pneumonia crepitations . bronchial breathing. immunocompromised patients likely nonimmunocompromised patients present pneumonia versus p .. patients pneumonia . uneventful recovery . recovered following complications . died pneumoniae infection . died due underlying comorbidities. patients died pneumoniae pneumonia comorbidities. conclusion results similar published data except finding infections common infants preschool children mortality rate pneumonia patients comorbidities high.
1,nitric oxide proinflammatory mediator lung disease,inflammatory diseases respiratory tract commonly associated elevated production nitric oxide increased indices dependent oxidative stress. although known antimicrobial antiinflammatory antioxidant properties various lines evidence support contribution lung injury several disease models. basis biochemical evidence often presumed dependent oxidations due formation oxidant peroxynitrite although alternative mechanisms involving phagocytederived heme proteins myeloperoxidase eosinophil peroxidase might operative conditions inflammation. overwhelming literature generation activities respiratory tract would beyond scope commentary review area comprehensively. instead focuses recent evidence concepts presumed contribution inflammatory diseases lung.
2,surfactant proteind pulmonary host defense,surfactant proteind spd participates innate response inhaled microorganisms organic antigens contributes immune inflammatory regulation within lung. spd synthesized secreted alveolar bronchiolar epithelial cells also expressed epithelial cells lining various exocrine ducts mucosa gastrointestinal genitourinary tracts. spd collagenous calciumdependent lectin collectin binds surface glycoconjugates expressed wide variety microorganisms oligosaccharides associated surface various complex organic antigens. spd also specifically interacts glycoconjugates molecules expressed surface macrophages neutrophils lymphocytes. addition spd binds specific surfactantassociated lipids influence organization lipid mixtures containing phosphatidylinositol vitro. consistent diverse vitro activities observation spddeficient transgenic mice show abnormal accumulations surfactant lipids respond abnormally challenge respiratory viruses bacterial lipopolysaccharides. phenotype macrophages isolated lungs spddeficient mice altered circumstantial evidence abnormal oxidant metabolism andor increased metalloproteinase expression contributes development emphysema. expression spd increased response many forms lung injury deficient accumulation appropriately oligomerized spd might contribute pathogenesis variety human lung diseases.
3,role endothelin lung disease,endothelin et amino acid peptide diverse biological activity implicated numerous diseases. et potent mitogen regulator smooth muscle tone inflammatory mediator may play key role diseases airways pulmonary circulation inflammatory lung diseases acute chronic. review focus biology et role lung disease.
4,gene expression epithelial cells response pneumovirus infection,respiratory syncytial virus rsv pneumonia virus mice pvm vi

In [62]:
data=df[['title', 'abstract']].apply(lambda x: '.'.join(x), axis=1)

In [63]:
data.head()

0                  clinical features cultureproven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia.objective retrospective chart review describes epidemiology clinical features patients cultureproven mycoplasma pneumoniae infections king abdulaziz university hospital jeddah saudi arabia. methods patients positive m. pneumoniae cultures respiratory specimens january december identified microbiology records. charts patients reviewed. results patients identified . required admission. infections . communityacquired. infection affected age groups common infants . preschool children .. occurred yearround common fall spring . threequarters patients . comorbidities. twentyfour isolates associated pneumonia upper respiratory tract infections bronchiolitis. cough . fever malaise . common symptoms crepitations wheezes common signs. patients pneumonia crepitations . bronchial breathing. immunocompromised patients likely nonimmunocompromised patients present 

In [64]:
tmp_corpus = data.map(lambda x: x.split('.'))

In [68]:
corpus = []
for i in tqdm(range(len(tmp_corpus))):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

100%|████████████████████████████████████████████████████████████████████████| 309325/309325 [00:32<00:00, 9654.42it/s]


In [69]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 3098860
Num of words - 29681668


In [70]:
phrases = Phrases(sentences=corpus,min_count=25,threshold=50)
bigram = Phraser(phrases)

2020-10-21 14:03:11,979 : INFO : collecting all words and their counts
2020-10-21 14:03:12,026 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2020-10-21 14:03:12,557 : INFO : PROGRESS: at sentence #10000, processed 112051 words and 96402 word types
2020-10-21 14:03:12,872 : INFO : PROGRESS: at sentence #20000, processed 218988 words and 170986 word types
2020-10-21 14:03:13,231 : INFO : PROGRESS: at sentence #30000, processed 330664 words and 243939 word types
2020-10-21 14:03:13,503 : INFO : PROGRESS: at sentence #40000, processed 420199 words and 300505 word types
2020-10-21 14:03:13,822 : INFO : PROGRESS: at sentence #50000, processed 526449 words and 363489 word types
2020-10-21 14:03:14,154 : INFO : PROGRESS: at sentence #60000, processed 629587 words and 423914 word types
2020-10-21 14:03:14,450 : INFO : PROGRESS: at sentence #70000, processed 724618 words and 479394 word types
2020-10-21 14:03:14,757 : INFO : PROGRESS: at sentence #80000, processed 818428 

2020-10-21 14:03:33,535 : INFO : PROGRESS: at sentence #740000, processed 6486516 words and 3022884 word types
2020-10-21 14:03:33,799 : INFO : PROGRESS: at sentence #750000, processed 6568401 words and 3050044 word types
2020-10-21 14:03:34,078 : INFO : PROGRESS: at sentence #760000, processed 6651877 words and 3078564 word types
2020-10-21 14:03:34,364 : INFO : PROGRESS: at sentence #770000, processed 6736350 words and 3106850 word types
2020-10-21 14:03:34,618 : INFO : PROGRESS: at sentence #780000, processed 6819567 words and 3134144 word types
2020-10-21 14:03:34,884 : INFO : PROGRESS: at sentence #790000, processed 6901131 words and 3161238 word types
2020-10-21 14:03:35,154 : INFO : PROGRESS: at sentence #800000, processed 6984093 words and 3188287 word types
2020-10-21 14:03:35,424 : INFO : PROGRESS: at sentence #810000, processed 7065049 words and 3214425 word types
2020-10-21 14:03:35,705 : INFO : PROGRESS: at sentence #820000, processed 7149505 words and 3242526 word types
2

2020-10-21 14:03:55,277 : INFO : PROGRESS: at sentence #1480000, processed 13248274 words and 4892635 word types
2020-10-21 14:03:55,639 : INFO : PROGRESS: at sentence #1490000, processed 13353945 words and 4911053 word types
2020-10-21 14:03:56,023 : INFO : PROGRESS: at sentence #1500000, processed 13461779 words and 4930955 word types
2020-10-21 14:03:56,379 : INFO : PROGRESS: at sentence #1510000, processed 13564140 words and 4948245 word types
2020-10-21 14:03:56,733 : INFO : PROGRESS: at sentence #1520000, processed 13670583 words and 4966604 word types
2020-10-21 14:03:57,071 : INFO : PROGRESS: at sentence #1530000, processed 13776933 words and 4986032 word types
2020-10-21 14:03:57,403 : INFO : PROGRESS: at sentence #1540000, processed 13883857 words and 5003948 word types
2020-10-21 14:03:57,772 : INFO : PROGRESS: at sentence #1550000, processed 13991215 words and 5022682 word types
2020-10-21 14:03:58,109 : INFO : PROGRESS: at sentence #1560000, processed 14093410 words and 50

2020-10-21 14:04:20,379 : INFO : PROGRESS: at sentence #2210000, processed 20908412 words and 6140130 word types
2020-10-21 14:04:20,668 : INFO : PROGRESS: at sentence #2220000, processed 21006498 words and 6156860 word types
2020-10-21 14:04:20,992 : INFO : PROGRESS: at sentence #2230000, processed 21106979 words and 6173379 word types
2020-10-21 14:04:21,294 : INFO : PROGRESS: at sentence #2240000, processed 21206811 words and 6190448 word types
2020-10-21 14:04:21,596 : INFO : PROGRESS: at sentence #2250000, processed 21305195 words and 6206207 word types
2020-10-21 14:04:21,899 : INFO : PROGRESS: at sentence #2260000, processed 21404143 words and 6221268 word types
2020-10-21 14:04:22,200 : INFO : PROGRESS: at sentence #2270000, processed 21503671 words and 6237902 word types
2020-10-21 14:04:22,522 : INFO : PROGRESS: at sentence #2280000, processed 21605284 words and 6254792 word types
2020-10-21 14:04:22,820 : INFO : PROGRESS: at sentence #2290000, processed 21703707 words and 62

2020-10-21 14:04:43,486 : INFO : PROGRESS: at sentence #2940000, processed 28128582 words and 7229391 word types
2020-10-21 14:04:43,875 : INFO : PROGRESS: at sentence #2950000, processed 28229357 words and 7242565 word types
2020-10-21 14:04:44,203 : INFO : PROGRESS: at sentence #2960000, processed 28328625 words and 7256029 word types
2020-10-21 14:04:44,481 : INFO : PROGRESS: at sentence #2970000, processed 28425342 words and 7269274 word types
2020-10-21 14:04:44,817 : INFO : PROGRESS: at sentence #2980000, processed 28521748 words and 7282637 word types
2020-10-21 14:04:45,142 : INFO : PROGRESS: at sentence #2990000, processed 28617766 words and 7295909 word types
2020-10-21 14:04:45,485 : INFO : PROGRESS: at sentence #3000000, processed 28716140 words and 7308490 word types
2020-10-21 14:04:45,817 : INFO : PROGRESS: at sentence #3010000, processed 28813797 words and 7321548 word types
2020-10-21 14:04:46,127 : INFO : PROGRESS: at sentence #3020000, processed 28907927 words and 73

In [71]:
for index,sentence in enumerate(corpus):
    corpus[index] = bigram[sentence]

In [72]:
# shuffle corpus
def shuffle_corpus(sentences):
    shuffled = list(sentences)
    random.shuffle(shuffled)
    return shuffled

In [73]:
# sg - skip gram |  window = size of the window | size = vector dimension
size = 100
window_size = 2 # sentences weren't too long, so
epochs = 100
min_count = 2
workers = 4
# train word2vec model using gensim
model = Word2Vec(corpus, sg=1,window=window_size,size=size,
                 min_count=min_count,workers=workers,iter=epochs,sample=0.01)

2020-10-21 14:09:52,931 : INFO : collecting all words and their counts
2020-10-21 14:09:52,931 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-21 14:09:53,018 : INFO : PROGRESS: at sentence #10000, processed 106275 words, keeping 15638 word types
2020-10-21 14:09:53,059 : INFO : PROGRESS: at sentence #20000, processed 207061 words, keeping 23072 word types
2020-10-21 14:09:53,119 : INFO : PROGRESS: at sentence #30000, processed 312344 words, keeping 29818 word types
2020-10-21 14:09:53,144 : INFO : PROGRESS: at sentence #40000, processed 396940 words, keeping 35770 word types
2020-10-21 14:09:53,209 : INFO : PROGRESS: at sentence #50000, processed 497155 words, keeping 40180 word types
2020-10-21 14:09:53,265 : INFO : PROGRESS: at sentence #60000, processed 594663 words, keeping 44528 word types
2020-10-21 14:09:53,310 : INFO : PROGRESS: at sentence #70000, processed 684293 words, keeping 48676 word types
2020-10-21 14:09:53,359 : INFO : PROGRESS: at 

2020-10-21 14:09:56,161 : INFO : PROGRESS: at sentence #720000, processed 5965482 words, keeping 186578 word types
2020-10-21 14:09:56,210 : INFO : PROGRESS: at sentence #730000, processed 6044740 words, keeping 187650 word types
2020-10-21 14:09:56,259 : INFO : PROGRESS: at sentence #740000, processed 6124599 words, keeping 188717 word types
2020-10-21 14:09:56,297 : INFO : PROGRESS: at sentence #750000, processed 6201560 words, keeping 189713 word types
2020-10-21 14:09:56,339 : INFO : PROGRESS: at sentence #760000, processed 6279940 words, keeping 190763 word types
2020-10-21 14:09:56,386 : INFO : PROGRESS: at sentence #770000, processed 6359476 words, keeping 191850 word types
2020-10-21 14:09:56,428 : INFO : PROGRESS: at sentence #780000, processed 6437583 words, keeping 192819 word types
2020-10-21 14:09:56,471 : INFO : PROGRESS: at sentence #790000, processed 6514277 words, keeping 193893 word types
2020-10-21 14:09:56,514 : INFO : PROGRESS: at sentence #800000, processed 659211

2020-10-21 14:09:59,396 : INFO : PROGRESS: at sentence #1430000, processed 11999955 words, keeping 258784 word types
2020-10-21 14:09:59,457 : INFO : PROGRESS: at sentence #1440000, processed 12100227 words, keeping 259684 word types
2020-10-21 14:09:59,515 : INFO : PROGRESS: at sentence #1450000, processed 12198834 words, keeping 260605 word types
2020-10-21 14:09:59,578 : INFO : PROGRESS: at sentence #1460000, processed 12297959 words, keeping 261739 word types
2020-10-21 14:09:59,632 : INFO : PROGRESS: at sentence #1470000, processed 12393617 words, keeping 262452 word types
2020-10-21 14:09:59,683 : INFO : PROGRESS: at sentence #1480000, processed 12495255 words, keeping 263387 word types
2020-10-21 14:09:59,738 : INFO : PROGRESS: at sentence #1490000, processed 12594912 words, keeping 264304 word types
2020-10-21 14:09:59,781 : INFO : PROGRESS: at sentence #1500000, processed 12696785 words, keeping 265201 word types
2020-10-21 14:09:59,839 : INFO : PROGRESS: at sentence #1510000,

2020-10-21 14:10:03,234 : INFO : PROGRESS: at sentence #2140000, processed 19087165 words, keeping 312319 word types
2020-10-21 14:10:03,280 : INFO : PROGRESS: at sentence #2150000, processed 19180159 words, keeping 312970 word types
2020-10-21 14:10:03,343 : INFO : PROGRESS: at sentence #2160000, processed 19272761 words, keeping 313696 word types
2020-10-21 14:10:03,396 : INFO : PROGRESS: at sentence #2170000, processed 19364900 words, keeping 314359 word types
2020-10-21 14:10:03,460 : INFO : PROGRESS: at sentence #2180000, processed 19459574 words, keeping 315028 word types
2020-10-21 14:10:03,523 : INFO : PROGRESS: at sentence #2190000, processed 19552419 words, keeping 315689 word types
2020-10-21 14:10:03,600 : INFO : PROGRESS: at sentence #2200000, processed 19644462 words, keeping 316258 word types
2020-10-21 14:10:03,664 : INFO : PROGRESS: at sentence #2210000, processed 19732250 words, keeping 316873 word types
2020-10-21 14:10:03,721 : INFO : PROGRESS: at sentence #2220000,

2020-10-21 14:10:07,012 : INFO : PROGRESS: at sentence #2850000, processed 25691022 words, keeping 353997 word types
2020-10-21 14:10:07,064 : INFO : PROGRESS: at sentence #2860000, processed 25783286 words, keeping 354503 word types
2020-10-21 14:10:07,092 : INFO : PROGRESS: at sentence #2870000, processed 25877012 words, keeping 355011 word types
2020-10-21 14:10:07,159 : INFO : PROGRESS: at sentence #2880000, processed 25971409 words, keeping 355512 word types
2020-10-21 14:10:07,191 : INFO : PROGRESS: at sentence #2890000, processed 26065607 words, keeping 356033 word types
2020-10-21 14:10:07,255 : INFO : PROGRESS: at sentence #2900000, processed 26168381 words, keeping 358536 word types
2020-10-21 14:10:07,301 : INFO : PROGRESS: at sentence #2910000, processed 26261537 words, keeping 359056 word types
2020-10-21 14:10:07,353 : INFO : PROGRESS: at sentence #2920000, processed 26354159 words, keeping 359516 word types
2020-10-21 14:10:07,392 : INFO : PROGRESS: at sentence #2930000,

...

2020-10-21 16:08:53,730 : INFO : EPOCH 96 - PROGRESS: at 71.31% examples, 379183 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:08:54,748 : INFO : EPOCH 96 - PROGRESS: at 72.69% examples, 379385 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:08:55,777 : INFO : EPOCH 96 - PROGRESS: at 74.06% examples, 379579 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:08:56,790 : INFO : EPOCH 96 - PROGRESS: at 75.37% examples, 379463 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:08:57,797 : INFO : EPOCH 96 - PROGRESS: at 76.77% examples, 379744 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:08:58,816 : INFO : EPOCH 96 - PROGRESS: at 78.20% examples, 380118 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:08:59,820 : INFO : EPOCH 96 - PROGRESS: at 79.54% examples, 380221 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:09:00,831 : INFO : EPOCH 96 - PROGRESS: at 80.70% examples, 379216 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:09:01,838 : INFO : EPOCH 96 - PROGRESS: at 81.76% examples, 377952 words/s

2020-10-21 16:10:03,772 : INFO : EPOCH 97 - PROGRESS: at 64.44% examples, 365997 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:10:04,771 : INFO : EPOCH 97 - PROGRESS: at 65.64% examples, 366041 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:10:05,780 : INFO : EPOCH 97 - PROGRESS: at 66.90% examples, 366615 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:10:06,823 : INFO : EPOCH 97 - PROGRESS: at 68.17% examples, 367033 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:10:07,832 : INFO : EPOCH 97 - PROGRESS: at 69.59% examples, 367753 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:10:08,843 : INFO : EPOCH 97 - PROGRESS: at 70.98% examples, 368240 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:10:09,862 : INFO : EPOCH 97 - PROGRESS: at 72.34% examples, 368475 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:10:10,925 : INFO : EPOCH 97 - PROGRESS: at 73.72% examples, 368594 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:10:11,927 : INFO : EPOCH 97 - PROGRESS: at 75.13% examples, 369244 words/s

2020-10-21 16:11:13,019 : INFO : EPOCH 98 - PROGRESS: at 60.15% examples, 379714 words/s, in_qsize 8, out_qsize 1
2020-10-21 16:11:14,062 : INFO : EPOCH 98 - PROGRESS: at 61.41% examples, 379769 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:11:15,068 : INFO : EPOCH 98 - PROGRESS: at 62.48% examples, 378576 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:11:16,065 : INFO : EPOCH 98 - PROGRESS: at 63.72% examples, 378771 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:11:17,081 : INFO : EPOCH 98 - PROGRESS: at 65.00% examples, 379056 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:11:18,097 : INFO : EPOCH 98 - PROGRESS: at 66.24% examples, 379166 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:11:19,105 : INFO : EPOCH 98 - PROGRESS: at 67.45% examples, 379291 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:11:20,105 : INFO : EPOCH 98 - PROGRESS: at 68.72% examples, 379264 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:11:21,109 : INFO : EPOCH 98 - PROGRESS: at 70.12% examples, 379597 words/s

2020-10-21 16:12:22,371 : INFO : EPOCH 99 - PROGRESS: at 57.58% examples, 391028 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:23,394 : INFO : EPOCH 99 - PROGRESS: at 58.93% examples, 391219 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:24,407 : INFO : EPOCH 99 - PROGRESS: at 60.21% examples, 391294 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:25,412 : INFO : EPOCH 99 - PROGRESS: at 61.51% examples, 391678 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:26,410 : INFO : EPOCH 99 - PROGRESS: at 62.86% examples, 392257 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:27,417 : INFO : EPOCH 99 - PROGRESS: at 64.13% examples, 392352 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:28,445 : INFO : EPOCH 99 - PROGRESS: at 65.45% examples, 392430 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:29,468 : INFO : EPOCH 99 - PROGRESS: at 66.71% examples, 392483 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:12:30,470 : INFO : EPOCH 99 - PROGRESS: at 68.01% examples, 392717 words/s

2020-10-21 16:13:31,991 : INFO : EPOCH 100 - PROGRESS: at 53.45% examples, 376987 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:13:33,034 : INFO : EPOCH 100 - PROGRESS: at 54.61% examples, 375900 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:13:34,059 : INFO : EPOCH 100 - PROGRESS: at 55.73% examples, 374727 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:13:35,061 : INFO : EPOCH 100 - PROGRESS: at 56.89% examples, 374267 words/s, in_qsize 8, out_qsize 0
2020-10-21 16:13:36,074 : INFO : EPOCH 100 - PROGRESS: at 58.06% examples, 373705 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:13:37,164 : INFO : EPOCH 100 - PROGRESS: at 59.09% examples, 371732 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:13:38,155 : INFO : EPOCH 100 - PROGRESS: at 60.28% examples, 371612 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:13:39,185 : INFO : EPOCH 100 - PROGRESS: at 61.51% examples, 371866 words/s, in_qsize 7, out_qsize 0
2020-10-21 16:13:40,218 : INFO : EPOCH 100 - PROGRESS: at 62.80% examples, 37210

In [74]:
# save model
model.save('w2v_model')

2020-10-21 16:15:19,910 : INFO : saving Word2Vec object under w2v_model, separately None
2020-10-21 16:15:20,110 : INFO : storing np array 'vectors' to w2v_model.wv.vectors.npy
2020-10-21 16:15:21,069 : INFO : not storing attribute vectors_norm
2020-10-21 16:15:21,113 : INFO : storing np array 'syn1neg' to w2v_model.trainables.syn1neg.npy
2020-10-21 16:15:22,137 : INFO : not storing attribute cum_table
2020-10-21 16:15:23,622 : INFO : saved w2v_model


In [75]:
# load word2vec model
model = Word2Vec.load('w2v_model')

2020-10-21 16:15:46,221 : INFO : loading Word2Vec object from w2v_model
2020-10-21 16:15:48,176 : INFO : loading wv recursively from w2v_model.wv.* with mmap=None
2020-10-21 16:15:48,176 : INFO : loading vectors from w2v_model.wv.vectors.npy with mmap=None
2020-10-21 16:15:48,700 : INFO : setting ignored attribute vectors_norm to None
2020-10-21 16:15:48,700 : INFO : loading vocabulary recursively from w2v_model.vocabulary.* with mmap=None
2020-10-21 16:15:48,700 : INFO : loading trainables recursively from w2v_model.trainables.* with mmap=None
2020-10-21 16:15:48,700 : INFO : loading syn1neg from w2v_model.trainables.syn1neg.npy with mmap=None
2020-10-21 16:15:48,781 : INFO : setting ignored attribute cum_table to None
2020-10-21 16:15:48,783 : INFO : loaded w2v_model


In [81]:
model.most_similar('coronavirus')

[('cornonavirus', 0.7682715058326721),
 ('corona', 0.7544320821762085),
 ('betacoronavirus', 0.6930726766586304),
 ('sarscoronavirus', 0.6911344528198242),
 ('conoravirus', 0.6673118472099304),
 ('cororavirus', 0.6667948365211487),
 ('cov', 0.6664053201675415),
 ('coranovirus', 0.6655673980712891),
 ('corononavirus', 0.6652964949607849),
 ('corovonavirus', 0.6527562141418457)]